In [1]:
import os
import arcpy
#import arcgis
import numpy as np
import torch
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import pylab as plt
from numpy import load
from itertools import combinations
from numpy import save
from dateutil import parser

%matplotlib inline

ImportError: DLL load failed while importing _arcgisscripting: The specified module could not be found.

In [2]:
prjPath=r'C:\projects\ESRICrash\roadData.gdb'
roadEdges=os.path.join(prjPath,'road_arterial__FeatureToLine') #can also prjPath+'\\Crash_new'
roadNodes =os.path.join(prjPath,'road_arterial_nodes')
roadmidpts=os.path.join(prjPath,'road_arterial_midpt')
roadSpatial=os.path.join(prjPath,'nodes_spatial_arterial')
edgeLinksFC=os.path.join(prjPath,'edgeLinks')
crashpoints=os.path.join(prjPath,'crash_ingrid_join_lines')
arcpy.env.workspace=prjPath
arcpy.env.overwriteOutput=True

srcFld = "src"
desFld = "des"
#fieldScale = 0
# Add src and des fields on the road edges
#arcpy.AddField_management(roadEdges, srcFld, "SHORT")
#arcpy.AddField_management(roadEdges, desFld, "SHORT")
# Enter for loop for each feature

NameError: name 'arcpy' is not defined

The following is to create all combinations of the edge index as node connections from src node to the des node.
The edgeindex will be used to create graph adjacematrix. The node is actually a mid point of the road line

In [84]:
edgeLinks =[]
for m in range(166):
    #delimitFld=arcpy.AddFieldDelimiters(roadNodes, "OID")
    with arcpy.da.SearchCursor(
        roadSpatial, 
        ["OID@","TARGET_FID","JOIN_FID"], 
        "TARGET_FID="+str(m+1)
    ) as cur:
        nodeLinks=[] # these are the links that are shared the same intersection
        for row in cur:
            nodeLinks.append(row[2])
            #print (row[2])
        if len(nodeLinks)>1:
            edgeLinks.append(nodeLinks)
mapping = {i+1: e for i, e in enumerate(edgeLinks)}
#print (mapping)
edgeindexList=[]
#edgeindexList =[combinations(c,2) for k,c in mapping.items()]
    # edgeindexList.append(comb)
for k,edgelink in mapping.items():
    edgeindex =[c for c in combinations(edgelink, 2)]# use combination function to get possible links
    edgeindexList.append(edgeindex)
#print (edgeindexList)

# Flatten the list
flat_list = [item for sublist in edgeindexList for item in sublist]
#print (flat_list)
# Extract the first and second elements of each tuple and store them in separate lists

srcrow = [x[0] for x in flat_list]
desrow = [x[1] for x in flat_list]
nodemapping={i+1:i for i in range(245)}
src=[nodemapping[s] for s in srcrow ]
des=[nodemapping[d] for d in desrow ]

# Create a torch.Tensor with shape (2, 5) containing the flattened values
edge_index = torch.tensor([src + des],dtype=torch.int32).view(2, -1)
#edge_index = edge_index.int()
# Print the result
print(edge_index)
 # this map the node index to the node line. they are one minus number of the index

tensor([[  0,   0,   1,  ..., 240, 240, 241],
        [  1,   2,   2,  ..., 241, 242, 242]], dtype=torch.int32)


In [85]:
from numpy import save
save('trainingData/edgeindex.npy', edge_index)


In [86]:
from numpy import load
# load array
edgeindex = load('trainingData/edgeindex.npy')
print (edgeindex)

[[  0   0   1 ... 240 240 241]
 [  1   2   2 ... 241 242 242]]


The following part is to insert edgeLink in arcgis Pro using src and des index. This create all the links from road to the
all connected road in the intersections.

In [ ]:
#mid points fields to search
fieldsmpt=["SHAPE@", "OID@"]
#edge links feature fields to insert polyline
fields = ["SHAPE@", "srcroad", "desroad"]

with arcpy.da.InsertCursor(edgeLinksFC,fields) as incursor:
# get the each link src and des nodes
#declare insert cursor
    for sd in range(len(srcrow)):
        srcnodeid=srcrow[sd]
        desnodeid=desrow[sd]
    # use an SQL query ("OBJECTID=srcnodeid" or "OJBECTID=desnodeid")
        sql_query="OBJECTID="+str(srcnodeid)+" or "+"OBJECTID="+str(desrow[sd])
        rds = [list(row) for row in arcpy.da.SearchCursor(roadmidpts, fieldsmpt, sql_query)]
        srcPtshp=rds[0][0].getPart()
        desPtshp=rds[1][0].getPart()
        # build array for line segment
        arrayPts = arcpy.Array()
        arrayPts.add(srcPtshp)
        arrayPts.add(desPtshp)
        # Create a Polyline object based on the array of points
        polyline = arcpy.Polyline(arrayPts)
        # Insert the feature
        incursor.insertRow([polyline,srcnodeid,desnodeid])
del incursor

Let's look at how the road links (edge links) look.

In [4]:
from arcgis.gis import GIS
#from arcgis.widgets import MapView
gis = GIS()
# load your feature class into a spatially enabled dataframe (sdf)
m_sdf = pd.DataFrame.spatial.from_featureclass(edgeLinksFC)
#m_map
m_map = gis.map(location = 'Marietta, GA', zoomlevel = 10)
m_sdf.spatial.plot(map_widget = m_map,
                renderer_type='s',
                cmap='Blues_r',  # use a different color map
                symbol_type='simple'               
                )
m_map

MapView(layout=Layout(height='400px', width='100%'))

load crash data

In [3]:
dfs=arcgis.features.GeoAccessor.from_featureclass(os.path.join(prjPath,crashpoints))
dfs.to_csv('trainingData/roadcrash.csv')

In [4]:
df = pd.read_csv('trainingData/roadcrash.csv')
df.head()

,Unnamed: 0,OBJECTID,ACCIDENTDATE,WEATHER,curv,Avg_Slope,DOW,SUM_LENGTH,MAX_SPEED_LIMIT,FIRST_LANE_TYPE,nodeid,SHAPE
0,0,1,2018-03-18 23:10:00.000001,3.0,1.141190,7.858093,NaN,17674.49719,45,1,78,"{'x': 2200251.299880162, 'y': 1422066.99999782..."
1,1,2,2018-03-18 21:56:00.000001,2.0,1.012250,9.580918,NaN,44164.68522,45,2,39,"{'x': 2166057.800025828, 'y': 1405272.50012232..."
2,2,3,2018-03-18 17:02:00.000000,1.0,1.021937,19.501251,NaN,45006.47942,65,2,13,"{'x': 2197852.34962558, 'y': 1390927.089508742..."
3,3,4,2018-03-18 17:27:00.000000,1.0,1.021937,19.501251,NaN,45006.47942,65,2,13,"{'x': 2198188.9316295795, 'y': 1391296.8066170..."
4,4,5,2018-03-18 14:30:00.000000,1.0,1.435386,23.058257,NaN,5106.35486,45,2,3,"{'x': 2175332.971107997, 'y': 1374303.37472473..."


In [5]:
df.isnull().sum()

Unnamed: 0             0
OBJECTID               0
ACCIDENTDATE           0
WEATHER             7896
curv                 146
Avg_Slope            146
DOW                61269
SUM_LENGTH             0
MAX_SPEED_LIMIT        0
FIRST_LANE_TYPE        0
nodeid                 0
SHAPE                  0
dtype: int64

In [32]:
df=df.fillna(0)

In [6]:
df = df.loc[:,["ACCIDENTDATE","DOW","nodeid"]]
df.head()

,ACCIDENTDATE,DOW,nodeid
0,2018-03-18 23:10:00.000001,NaN,78
1,2018-03-18 21:56:00.000001,NaN,39
2,2018-03-18 17:02:00.000000,NaN,13
3,2018-03-18 17:27:00.000000,NaN,13
4,2018-03-18 14:30:00.000000,NaN,3


In [21]:
#add weekday of the crash
def week_day(DOW):
        if datetime.strptime(DOW[0:-7],'%Y-%m-%d %H:%M:%S').weekday() < 5:
            return 1
        else:
            return 0

In [22]:
df['DOW'] = df.apply(lambda x: week_day(x['ACCIDENTDATE']), axis=1)

In [26]:
#save to a .npy file so we don't need to run every time with notebook
crashnpy=df.to_numpy()
save('trainingData/crash.npy', crashnpy)

In [57]:
# load array
crashdata = load('trainingData/crash.npy', allow_pickle=True)
print (crashdata.shape)
print (crashdata[0,:])

(61269, 3)
['2018-03-18 23:10:00.000001' 0 78]


Load node features, basically it is a road feature since we represent road midpoints as nodes

In [5]:
#curvature
curv=\
"""
import math
def curv(shp):
  x1=shp.firstPoint.x
  y1=shp.firstPoint.y
  x2=shp.lastPoint.x
  y2=shp.lastPoint.y
  
  eucliDistance=math.sqrt((x2-x1)**2+(y2-y1)**2)
  l=shp.length
  if eucliDistance>0:
     return l/eucliDistance
  return 1.0
"""

In [7]:
#add curvature field to the CobbRoads fc
curvfld=[['curv','double']]
_=arcpy.management.AddFields(roadEdges,curvfld)
_=arcpy.management.CalculateField(roadEdges,'curv','curv(!Shape!)', code_block=curv)

In [28]:
dfrds=arcgis.features.GeoAccessor.from_featureclass(os.path.join(prjPath,roadEdges))
dfrds.to_csv('trainingData/roadedges.csv')

In [38]:
dfre = pd.read_csv('trainingData/roadedges.csv')
dfre.head()

,Unnamed: 0,OBJECTID,MAX_SPEED_LIMIT,FIRST_LANE_TYPE,src,des,nodeid,curv,length,SHAPE
0,0,1,60,2,1,2,1,1.003295,8037.337567,"{'hasM': True, 'paths': [[[2184682.8000066876,..."
1,1,2,60,2,2,3,2,1.000010,12765.986345,"{'hasM': True, 'paths': [[[2176914.499967452, ..."
2,2,3,45,2,2,4,3,1.066625,5106.354857,"{'hasM': True, 'paths': [[[2176914.499967452, ..."
3,3,4,45,1,7,5,4,1.028404,12522.000415,"{'hasM': True, 'paths': [[[2174871.799933549, ..."
4,4,5,40,1,6,5,5,1.019563,9874.799060,"{'hasM': True, 'paths': [[[2191080.799884647, ..."


In [30]:
#drop these fields
dfre=dfre.drop(columns=['src','des','nodeid'])

In [32]:
#these fields need to be scaled using scikit-learn's minmaxScaler
scale_fld=['MAX_SPEED_LIMIT','FIRST_LANE_TYPE','curv','length']
scaler = MinMaxScaler()
scale_values = dfre.loc[:,scale_fld]
scale_values = scaler.fit_transform(scale_values)
dfre.loc[:,scale_fld]=scale_values

In [33]:
dfnodes=dfre.to_numpy()
dfnodes=dfnodes[:,2:6]
dfnodes.shape
print (dfnodes)

[[0.8333333333333333 1.0 0.007756406427215978 0.32105720730318105]
 [0.8333333333333333 1.0 0.0 0.5110078216472885]
 [0.33333333333333326 1.0 0.15729063345091676 0.20331914034318951]
 [0.33333333333333326 0.0 0.06704299737279173 0.5012068659083802]
 [0.16666666666666652 0.0 0.04616844407662413 0.39486834356477685]
 [0.16666666666666652 0.0 0.19402324701525586 0.37806874380811617]
 [0.33333333333333326 0.0 0.07144567197626195 0.24572029735997974]
 [0.33333333333333326 1.0 0.02968685224842238 0.22454698850539415]
 [0.33333333333333326 0.0 0.06525038147180107 0.27419100759254733]
 [0.33333333333333326 0.0 0.11759042281822962 0.8455345802511769]
 [0.33333333333333326 0.0 0.08017603141167262 0.5239354284649587]
 [0.16666666666666652 0.0 0.03169040380681576 0.09793451438474156]
 [0.9999999999999998 1.0 0.37098084986798296 0.3833953392521733]
 [0.6666666666666665 0.0 0.0023957419867963736 0.20868074969389747]
 [0.16666666666666652 0.0 0.05844462447564247 0.47906766113912397]
 [0.3333333333333

In [35]:
#now we have four features of nodes for graph, that's save it to npy file so no need to process every time
save('trainingData/nodefeatures.npy', dfnodes)

In [6]:
# load array
nodesdata = load('trainingData/nodefeatures.npy', allow_pickle=True)
print (nodesdata[:5,:])

[[0.8333333333333333 1.0 0.007756406427215978 0.32105720730318105]
 [0.8333333333333333 1.0 0.0 0.5110078216472885]
 [0.33333333333333326 1.0 0.15729063345091676 0.20331914034318951]
 [0.33333333333333326 0.0 0.06704299737279173 0.5012068659083802]
 [0.16666666666666652 0.0 0.04616844407662413 0.39486834356477685]]


In [7]:
#time period to contruct time series data
begin=datetime.strptime('2018-03-18 00:00:00','%Y-%m-%d %H:%M:%S')
end  =datetime.strptime('2021-03-30 00:00:00','%Y-%m-%d %H:%M:%S')

In [8]:
def return_time_index(time_stamp, start, end):
    if time_stamp<start or time_stamp > end:
        return -1
    index =int((time_stamp-start).days)
    return index
diff=int((end-begin).days)+1
diff

1109

In [9]:
new_nodesdata = np.pad(nodesdata, ((0, 0), (0, 2)), mode='constant')#pad two new column with o for DOW and accidents
new_nodesdata.shape

(245, 6)

In [55]:
print (new_nodesdata[:5,:])

[[0.8333333333333333 1.0 0.007756406427215978 0.32105720730318105 1 0]
 [0.8333333333333333 1.0 0.0 0.5110078216472885 1 0]
 [0.33333333333333326 1.0 0.15729063345091676 0.20331914034318951 1 0]
 [0.33333333333333326 0.0 0.06704299737279173 0.5012068659083802 1 0]
 [0.16666666666666652 0.0 0.04616844407662413 0.39486834356477685 1 0]]


In [12]:
#generate graph road matrix (1109x245x6) 1109 time series 245 nodes 6 node features
graphMatrix=np.zeros((diff,nodesdata.shape[0],6))

In [54]:
graphMatrix.shape

(1109, 245, 6)

In [14]:
# start date
# difference between each date. D means one day
D = 'D'

date_list = pd.date_range(begin, end, freq=D)
print(f"Creating list of dates starting from {begin} to {end}")
print(len(date_list))

# if you want dates in string format then convert it into string
#print(date_list.strftime("%Y-%m-%d"))

Creating list of dates starting from 2018-03-18 00:00:00 to 2021-03-30 00:00:00
1109


In [15]:
#add weekday of the crash
def week_day_d(DOW):
        if DOW.weekday() < 5:
            return 1
        else:
            return 0

In [28]:
dowmapping= [week_day_d(d) for d in date_list]

In [31]:
dowmapping[1]

1

In [45]:
#assign the dow column as dow (1 or 0 ) 
new_nodesdata[:,-2]=dowmapping[1]

In [46]:
print (new_nodesdata[0]) # it is correct

[0.8333333333333333 1.0 0.007756406427215978 0.32105720730318105 1 0]


In [40]:
#fill each node into the time sequence
for idate in range(diff):
    new_nodesdata[:,-2]=dowmapping[idate]#this assgin all dow for that date index
    graphMatrix[idate,:]=new_nodesdata

In [49]:
graphMatrix[10,1]

array([0.83333333, 1.        , 0.        , 0.51100782, 1.        ,
       0.        ])

In [51]:
dftestindex=parser.parse('2018-03-20 23:10:00.000001')
idx=return_time_index(dftestindex, begin, end)
idx

2

In [87]:
#Now let's create graph dataset (1109,245,6) by dropping the accidents to each node
xData=graphMatrix.copy()
accident=np.zeros((diff,245))
y=np.zeros((diff,245))
for ix in range(crashdata.shape[0]):
    dftestindex=parser.parse(crashdata[ix][0]) #crashdata[0] is the accidentdate column
    idx=return_time_index(dftestindex, begin, end)
    dfcrashlist=[]
    if idx>=0:
        nodeidx=nodemapping[crashdata[ix][2]]
        accident[idx][nodeidx]=accident[idx][nodeidx]+1
        dfcrashlist=[accident[idx][nodeidx]]
        vectorcrash=np.asarray(dfcrashlist)
        vector=graphMatrix[idx][nodeidx][0:5]
        vector =np.concatenate((vector,vectorcrash),axis=None)
        xData[idx][nodeidx]=vector
        y[idx][nodeidx]=accident[idx][nodeidx]
       # print (dftestindex,gridx,  testData[idx][gridx])   
#print ("done", xData[0,0])

In [65]:
crashdata[0][2]

78

In [88]:
print ("done", xData[1,0])

done [0.83333333 1.         0.00775641 0.32105721 1.         1.        ]


In [99]:
save('trainingData/xdata.npy', xData)

In [101]:
gdata = load('trainingData/xdata.npy', allow_pickle=True)

In [102]:
print (gdata[1,0])

[0.83333333 1.         0.00775641 0.32105721 1.         1.        ]
